In [3]:
# %pip install --quiet langchain_community

In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/123Accu.txt")
data = loader.load()

In [5]:
# data[0].metadata
# {'source': 'data/123Accu.txt'}

# data[0].page_content
# 'Supplier Snapshot Strategy\n123Accu B.V. is headquart...

In [6]:
# %pip install --quiet unstructured

In [7]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('./data', glob="**/*.txt", show_progress=True)

docs = loader.load()
docs

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:02<00:00,  1.11it/s]


[Document(page_content='Supplier Snapshot Strategy 123Accu B.V. is headquartered in Almere, Netherlands and active in the sector: Wholesale Electronic Markets and Agents and Brokers. We have 1 spend(s) with 123Accu B.V. adding up to a volume of USD 665.36 representing 0.000% of the total analyzed spend on the ADA Platform. We transact with the supplier in the following currency(ies): EUR. Our average payment terms with the supplier are 0 days, which are 27 days lower compared to the average payment terms of the total analyzed spend. On this particular spend (\'Spend 1\') of USD 665.36, based on the analysis performed by the ADA Platform with a moderate approach, we recommend extending our current payment terms with the supplier by 45 days. The additional 45 days extension with the supplier moves our terms to 45 days, which represents USD 82.26 of working capital optimization for our organization. 123Accu B.V. is a Micro Enterprise with approximately USD 2.3m in estimated annual sales. 

In [8]:
# %pip install --quiet langchain

## CharacterTextSplitter

In [9]:
"""
Split by character
This is the simplest method. This splits based on characters (by default “”) and measure chunk length by number of characters.
"""

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

## RecursiveCharacterTextSplitter

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=300,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

In [11]:
texts = text_splitter.create_documents([docs[0].page_content])
# for doc in docs:
#     texts = text_splitter.create_documents(docs)
    
texts

[Document(page_content='Supplier Snapshot Strategy 123Accu B.V. is headquartered in Almere, Netherlands and active in the sector: Wholesale Electronic Markets and Agents and Brokers. We have 1 spend(s) with 123Accu B.V. adding up to a volume of USD 665.36 representing 0.000% of the total analyzed spend on the ADA Platform.'),
 Document(page_content="of the total analyzed spend on the ADA Platform. We transact with the supplier in the following currency(ies): EUR. Our average payment terms with the supplier are 0 days, which are 27 days lower compared to the average payment terms of the total analyzed spend. On this particular spend ('Spend 1')"),
 Document(page_content="spend. On this particular spend ('Spend 1') of USD 665.36, based on the analysis performed by the ADA Platform with a moderate approach, we recommend extending our current payment terms with the supplier by 45 days. The additional 45 days extension with the supplier moves our terms to 45 days,"),
 Document(page_content=

## Add metadata

In [12]:
# Metadata
# texts[0].metadata = {'title': "123Accu", 'source': "123Accu.txt"}

In [13]:
for t in texts:
    t.metadata = docs[0].metadata

texts

[Document(page_content='Supplier Snapshot Strategy 123Accu B.V. is headquartered in Almere, Netherlands and active in the sector: Wholesale Electronic Markets and Agents and Brokers. We have 1 spend(s) with 123Accu B.V. adding up to a volume of USD 665.36 representing 0.000% of the total analyzed spend on the ADA Platform.', metadata={'source': 'data\\123Accu.txt'}),
 Document(page_content="of the total analyzed spend on the ADA Platform. We transact with the supplier in the following currency(ies): EUR. Our average payment terms with the supplier are 0 days, which are 27 days lower compared to the average payment terms of the total analyzed spend. On this particular spend ('Spend 1')", metadata={'source': 'data\\123Accu.txt'}),
 Document(page_content="spend. On this particular spend ('Spend 1') of USD 665.36, based on the analysis performed by the ADA Platform with a moderate approach, we recommend extending our current payment terms with the supplier by 45 days. The additional 45 day

In [14]:
texts[0].metadata

{'source': 'data\\123Accu.txt'}

## Embeddings

In [15]:
# %pip install sentence_transformers

In [16]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

d:\GitHub\llm-financials\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# query_result = embeddings.embed_query(texts[0].page_content)
# query_result

## Chroma

In [18]:
# %pip install --quiet chromadb

In [19]:
from langchain_community.vectorstores import Chroma

# persist_directory="./chroma_db"
vectorstore = Chroma.from_documents(texts, embeddings)

# results = vectorstore.similarity_search("What is the supplier?")
# results[0]

## FAISS

In [20]:
# %pip install --quiet faiss-cpu

In [21]:
# from langchain_community.vectorstores import FAISS

In [22]:
# Load env variables
from dotenv import load_dotenv

load_dotenv()

True

## LLM

In [23]:
from langchain_community.llms import HuggingFaceHub

repo_id = "google/flan-t5-xxl"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

# repo_id=repo_id, model_kwargs={"temperature": 0.7, "max_length": 50}
llm = HuggingFaceHub(
    repo_id=repo_id
)

## RetrievalQA

In [24]:
# from langchain.chains import RetrievalQA

# chain = RetrievalQA.from_llm(
#     llm=llm, retriever=vectorstore.as_retriever()
# )

# chain

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=HuggingFaceHub(client=<InferenceClient(model='google/flan-t5-xxl', timeout=None)>, repo_id='google/flan-t5-xxl', task='text2text-generation')), document_prompt=PromptTemplate(input_variables=['page_content'], template='Context:\n{page_content}'), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001FEC675F950>))

In [25]:
# query = "What is the supplier?"
# # 'result': '123Accu B.V.'

# chain.invoke({"query": query})

KeyboardInterrupt: 

In [ ]:
query = "What is the supplier location?"
# 'result': 'Almere, Netherlands'

chain.invoke({"query": query})

{'query': 'What is the supplier location?', 'result': 'Almere, Netherlands'}

In [ ]:
query = "What is the supplier economic sector?"
# 'result': '123Accu B.V.'

chain.invoke({"query": query})

{'query': 'What is the supplier economic sector?', 'result': '123Accu B.V.'}

In [ ]:
query = "given a Pay on Terms Strategy, what is the ADA recommendation?"
# (OK) 'result': 'ADA cannot advise improving our payment terms based on the Supplier Parent Strategy as the information is not available.'}

chain({"query": query})

d:\GitHub\llm-financials\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'given a Pay on Terms Strategy, what is the ADA recommendation?',
 'result': 'ADA cannot advise improving our payment terms based on the Supplier Parent Strategy as the information is not available.'}

In [ ]:
query = "Using the Supplier Parent Strategy, how can we improve our payment terms?"
# (X) 'result': 'ADA cannot advise improving our payment terms based on the Supplier Parent Strategy as the information is not available.'

chain({"query": query})

{'query': 'Using the Supplier Parent Strategy, how can we improve our payment terms?',
 'result': 'ADA cannot advise improving our payment terms based on the Supplier Parent Strategy as the information is not available.'}

In [ ]:
query = "Using the Industry Group Strategy, how can we improve our payment terms?"
# (X)'result': 'ADA cannot advise improving our payment terms based on the Supplier Parent Strategy as the information is not available.'

query = "What information info do we have related to the Industry Group Strategy?"
# 'result': 'Industry Group Strategy 21 days 0 days DSO Strategy 0 days 0 days Financing Strategy 0 days 0 days Pay on Terms Strategy 0 days 0 days'

chain({"query": query})

{'query': 'What information info do we have related to the Industry Group Strategy?',
 'result': 'Industry Group Strategy 21 days 0 days DSO Strategy 0 days 0 days Financing Strategy 0 days 0 days Pay on Terms Strategy 0 days 0 days'}

In [ ]:
query = "Using the Cash Discount Conversion Strategy, how can we improve our payment terms?"
# 'result': 'By converting early cash discounts we pay 123Accu B.V. on this spend ("Spend 1") into longer payment terms.'

chain({"query": query})

{'query': 'Using the Cash Discount Conversion Strategy, how can we improve our payment terms?',
 'result': 'By converting early cash discounts we pay 123Accu B.V. on this spend ("Spend 1") into longer payment terms.'}

## RetrievalQAWithSourcesChain

In [26]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnableParallel

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
# llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )


rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [29]:
rag_chain_with_source.invoke("what is the strategy with the highest Proposed Payment Terms ?")

{'context': [Document(page_content='peer group has on average payment term 44 days. In comparison with our current average payment terms of this spend ("Spend 1") with our supplier, we pay 44 days earlier than the market average. Based on this strategy, we could extend our payment terms by 44 days to be aligned with the market and', metadata={'source': 'data\\123Accu.txt'}),
  Document(page_content='## Pay on Terms Strategy ADA cannot advise improving our payment terms based on the Supplier Parent Strategy as the information is not available.', metadata={'source': 'data\\123Accu.txt'}),
  Document(page_content='Strategy Name                       |Proposed Payment Terms       |Current Payment Terms ------------------------------------------------------------------|-------------------------- Supplier Snapshot Strategy          |45 days                      | 0 days Peer Group Average Strategy         |44', metadata={'source': 'data\\123Accu.txt'}),
  Document(page_content='## Financing 

In [30]:
rag_chain_with_source.invoke("what is the strategy with the lowest Proposed Payment Terms ?")

{'context': [Document(page_content='## Pay on Terms Strategy ADA cannot advise improving our payment terms based on the Supplier Parent Strategy as the information is not available.', metadata={'source': 'data\\123Accu.txt'}),
  Document(page_content='peer group has on average payment term 44 days. In comparison with our current average payment terms of this spend ("Spend 1") with our supplier, we pay 44 days earlier than the market average. Based on this strategy, we could extend our payment terms by 44 days to be aligned with the market and', metadata={'source': 'data\\123Accu.txt'}),
  Document(page_content='## Financing Strategy Unfortunately, there is no opportunity to optimize our payment terms based on the Financing, as the option is not available or is not selected.', metadata={'source': 'data\\123Accu.txt'}),
  Document(page_content='Strategy Name                       |Proposed Payment Terms       |Current Payment Terms --------------------------------------------------------

In [42]:
from langchain.chains import create_retrieval_chain 

chain = create_retrieval_chain(
    llm=llm, retriever=vectorstore.as_retriever()
)

chain

TypeError: create_retrieval_chain() got an unexpected keyword argument 'llm'

In [ ]:
%pip install --quiet langchainhub

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")


In [ ]:

retriever = vectorstore.as_retriever()

combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

chain.invoke({"question": "..."})

ValueError: Document prompt requires documents to have metadata variables: ['source']. Received document with missing metadata: ['source'].

In [9]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

In [16]:
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)
from langchain_community.chat_models.huggingface import ChatHuggingFace

messages = [
    SystemMessage(content="You're a helpful assistant. You have a Phd in Finance, and you are interested in cashflow. "),
    HumanMessage(
        content="What is cashflow?"
    ),
]

chat_model = ChatHuggingFace(llm=llm)

In [17]:
res = chat_model.invoke(messages)
print(res.content)

<|system|>
You're a helpful assistant. You have a Phd in Finance, and you are interested in cashflow. </s>
<|user|>
What is cashflow?</s>
<|assistant|>
Cashflow refers to the movement of cash in and out of a business or individual's financial position over a specific period of time. It represents the net amount of cash that is generated or used by an entity during that time, taking into account all cash inflows and outflows, such as revenue, expenses, investments, and financing activities. A positive cashflow indicates that more cash is coming in than going out, while a negative cashflow means that more cash is being used than generated. Cashflow is a critical financial metric that helps businesses and individuals manage their liquidity, make informed financial decisions, and avoid cash shortages.


In [3]:
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

KeyboardInterrupt: 

In [ ]:
text = "This is a test document."

query_result = embeddings.embed_query(text)

query_result[:3]

In [1]:
doc_result = embeddings.embed_documents([text])
doc_result

NameError: name 'embeddings' is not defined

In [2]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.llms import HuggingFaceHub

from langchain.prompts import PromptTemplate

from dotenv import load_dotenv

load_dotenv()

True

In [18]:
repo_id = "google/flan-t5-xxl"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

# repo_id=repo_id, model_kwargs={"temperature": 0.7, "max_length": 50}
llm = HuggingFaceHub(
    repo_id=repo_id
)

d:\anaconda3\envs\langchain\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


# Restaurant Name

In [20]:
restaurant_name_prompt_template = PromptTemplate(
    input_variables=["cuisine"],
    template= "I want to open a restaurant for {cuisine} cuisine. Suggest a good restaurant name."
)

restaurant_name_prompt_template_formatted = restaurant_name_prompt_template.format(cuisine="Chinese")
restaurant_name_prompt_template_formatted

'I want to open a restaurant for Chinese cuisine. Suggest a good restaurant name.'

In [21]:
from langchain.chains import LLMChain

restaurant_name_chain = LLMChain(
    prompt=restaurant_name_prompt_template, llm=llm
)

name_of_restaurant = restaurant_name_chain.invoke("Chinese")
name_of_restaurant

{'cuisine': 'Chinese', 'text': 'The Dragon'}

# Simple Seq Chain

In [30]:
food_items_prompt_template = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest three items to include in a menu for a restaurant called {restaurant_name}. Return the items as comma separated list"
)

food_items_chain = LLMChain(
    prompt=food_items_prompt_template, llm=llm, verbose=True
)

In [27]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains=[restaurant_name_chain, food_items_chain], verbose=True
)

menu_items = overall_chain.invoke("Chinese")
print(f"{menu_items['input']}:\n{menu_items['output']}")



> Entering new SimpleSequentialChain chain...
The Dragon


> Entering new LLMChain chain...
Prompt after formatting:
Suggest three items to include in a menu for a restaurant called The Dragon. Return the items as comma separated list

> Finished chain.
burger, ribs, fried chicken

> Finished chain.
Chinese:
burger, ribs, fried chicken


# Seq Chain

In [34]:
from langchain.chains import SequentialChain

restaurant_name_chain = LLMChain(
    prompt=restaurant_name_prompt_template, llm=llm, output_key="restaurant_name"
)

food_items_chain = LLMChain(
    prompt=food_items_prompt_template, llm=llm, verbose=True, output_key="food_items"
)

seq_chain = SequentialChain(chains=[restaurant_name_chain, food_items_chain], input_variables=["cuisine"], output_variables=["restaurant_name", "food_items"], verbose=True)

seq_chain({"cuisine": "American"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Suggest three items to include in a menu for a restaurant called The American. Return the items as comma separated list

> Finished chain.

> Finished chain.


{'cuisine': 'American',
 'restaurant_name': 'The American',
 'food_items': 'burger, ribs, steak'}

# Old

In [6]:
# Install required packages
# !pip install langchain
# !pip install chromadb
# !pip install lark
# !pip install openai
# !pip install tiktoken

import os
import argparse
import sys
import time
import re

# from langchain.llms import OpenAI
# from langchain.chat_models import ChatOpenAI
# from langchain.embeddings.openai import OpenAIEmbeddings

from langchain_community.llms import HuggingFaceHub

from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents import Tool, initialize_agent

In [2]:
# Define scripts and metadata
scripts = [
    {
        "id": "123Accu B.V.",
        "path": "data/script1_without_meta.txt",
        "inverse_map": {
            "05f10861-37db-4ef4-a0d2-24d222e6158d": "De Heus Animal Nutrition B.V.",
            "dfe452a3-a8c9-44f2-a65f-2d75e3fe9961": "123Accu B.V.",
            "db14ee98-88af-4696-b772-355fef3bd731": "All Suppliers - CF - No Financing"
        },
        "scope": {
            "company": "De Heus Animal Nutrition B.V.",
            "spendFileVersion": "De_Heus_20230427 (1).xlsx (Ver. 1)",
            "supplier": "123Accu B.V.",
            "program": "All Suppliers - CF - No Financing"
        },
        "meta": {
            "company": "05f10861-37db-4ef4-a0d2-24d222e6158d",
            "supplier": "dfe452a3-a8c9-44f2-a65f-2d75e3fe9961",
            "program": "db14ee98-88af-4696-b772-355fef3bd731"
        }
    },
    {
        "id": "Deutsche Post AG",
        "path": "data/script2_without_meta.txt",
        "inverse_map": {
            "5a73140d-14b5-4c3a-b62f-a8d7136f0e2d": "Adidas AG",
            "fd6ab5b2-9ad8-478f-a6b3-72d492e35dc0": "Deutsche Post AG",
            "b3320b0e-ae67-42ad-af1e-f2bda62f2db1": "Adidas Financing Program (60+ Days)"
        },
        "scope": {
            "company": "Adidas AG",
            "spendFileVersion": "Adidas_20220929.xlsx (Ver. 1)",
            "supplier": "Deutsche Post AG",
            "program": "Adidas Financing Program (60+ Days)"
        },
        "meta": {
            "company": "5a73140d-14b5-4c3a-b62f-a8d7136f0e2d",
            "supplier": "fd6ab5b2-9ad8-478f-a6b3-72d492e35dc0",
            "program": "b3320b0e-ae67-42ad-af1e-f2bda62f2db1"
        }
    },
    {
        "id": "Bel S.R.L.",
        "path": "data/script3_without_meta.txt",
        "inverse_map": {
            "5a73140d-14b5-4c3a-b62f-a8d7136f0e2d": "Adidas AG",
            "3a42cfac-c0e7-4aaa-af9f-28844d6fef09": "Bel S.R.L.",
            "b3320b0e-ae67-42ad-af1e-f2bda62f2db1": "Adidas Financing Program (60+ Days)"
        },
        "scope": {
            "company": "Adidas AG",
            "spendFileVersion": "Adidas_20220929.xlsx (Ver. 1)",
            "supplier": "Bel S.R.L.",
            "program": "Adidas Financing Program (60+ Days)"
        },
        "meta": {
            "company": "5a73140d-14b5-4c3a-b62f-a8d7136f0e2d",
            "supplier": "3a42cfac-c0e7-4aaa-af9f-28844d6fef09",
            "program": "b3320b0e-ae67-42ad-af1e-f2bda62f2db1"
        }
    }
]

In [4]:
all_docs = []
for script in scripts:
    loader = TextLoader(script["path"], encoding='utf8')
    docs = loader.load_and_split(
        CharacterTextSplitter(chunk_size=1000, chunk_overlap=100))

    for doc in docs:
        doc.metadata = script["meta"]

    all_docs.extend(docs)

embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(all_docs, embeddings)

Created a chunk of size 1597, which is longer than the specified 1000
Created a chunk of size 1574, which is longer than the specified 1000
Created a chunk of size 1575, which is longer than the specified 1000


ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:

metadata_field_info = [
    AttributeInfo(
        name="company",
        description="The company is represented by a universally unique identifier or UUID. The UUID is composed of 32 characters",
        type="string"
    ),
    AttributeInfo(
        name="supplier",
        description="The supplier is represented by a universally unique identifier or UUID. The UUID is composed of 32 characters",
        type="string"
    ),
    AttributeInfo(
        name="program",
        description="The program is represented by a universally unique identifier or UUID. The UUID is composed of 32 characters",
        type="string"
    )
]


# Create the retriever and memory objects
document_content_description = "Strategies used by companies to negotiate payment terms. The best strategies are those that provide the highest payment terms."

retriever = SelfQueryRetriever.from_llm(
    OpenAI(temperature=0),
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=False
)


memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True)
retriever_chain = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0), retriever, memory=memory)

tools = [
    Tool(
        name="Calculumm",
        func=retriever_chain.run,
        description=("Use this tool for answering questions about how to Improve Supplier Intelligence, Optimize Payment Terms, Unlock Working Capital, and Generate Cash Flow.")
    )
]


In [ ]:

llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model_name='gpt-3.5-turbo',
    temperature=0.6
)

memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=1,
    return_messages=True
)

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)